# Apache Cassandra use-case with music streaming app

## ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data looks like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [1]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [2]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_db
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Setting Keyspace

In [3]:
# Setting KEYSPACE to the keyspace specified above
try: 
    session.set_keyspace('sparkify_db')
except Exception as e:
    print(e)

### Creating tables to run the following queries with Apache Cassandra.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

SELECT artist, song, lenght FROM test_db WHERE sessionId = 338 AND itemInSession = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
SELECT artist, song, fisrtName, lastName FROM test_db WHERE userid = 10 AND sessionid = 182

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

SELECT fisrtName, lastName FROM test_db WHERE song = 'All Hands Against His Own'


## Task1: Getting the artist, song title and song length
For this query the following columns are considered: sessionId, itemInSession, artist, song, length. 
SessionId and itemInSession are used as a COMPOSITE PRIMARY KEY, where sessionId is a PARTITION KEY and itemInSession is a CLUSTERING COLUMN.
Artist, song, length columns are used to retrieve necessary data to satisfy the query.

In [99]:
query = "DROP TABLE IF EXISTS song_stats_by_artist"

try:
    session.execute(query)
except Exception as e:
    print(e)


query = "CREATE TABLE IF NOT EXISTS song_stats_by_artist"
query = query + """(
                    sessionId int,
                    itemInSession int,
                    artist text,
                    song text,
                    length text,
                    PRIMARY KEY 
                        (
                            sessionId, 
                            itemInSession
                        )
                    )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [100]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_stats_by_artist 
                                        (
                                            sessionId, 
                                            itemInSession,
                                            artist, 
                                            song, 
                                            length
                                        )
                                    """
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[5]))

#### Do a SELECT to verify that the data have been inserted into each table

In [101]:
query = "SELECT artist, song, length FROM song_stats_by_artist WHERE sessionId = 338 AND itemInSession = 4;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, '-', row.song, '-', row.length)

Faithless - Music Matters (Mark Knight Dub) - 495.3073


## Task2: Getting the playlist (artist and song title) by user
For this query the following columns are considered: userId, sessionId, itemInSession, artist, song, firstName, lastName.
UserId, sessionId and itemInSession are used as a COMPOSITE PRIMARY KEY, where userId and sessionId are used as a PARTITION KEY and itemInSession is a CLUSTERING COLUMN.
Artist, song, firstName and lastName columns are used to retrieve necessary data to satisfy the query.

In [82]:
query = "DROP TABLE IF EXISTS song_playlist_session"

try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE song_playlist_session" 
query = query + """(
                     userId int,
                     sessionId int,
                     itemInSession int,
                     artist text,
                     song text,
                     firstName text,
                     lastName text,
                     PRIMARY KEY ((userId, sessionId), itemInSession)
                     )"""



try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_playlist_session 
                                        (
                                            userId,
                                            sessionId,
                                            itemInSession,
                                            artist,
                                            song,
                                            firstName,
                                            lastName
                                        )"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0],  line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [83]:
query = "SELECT artist, song, firstname, lastname FROM song_playlist_session WHERE userid = 10 AND sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, '-', row.song, '-', row.firstname, row.lastname)

Down To The Bone - Keep On Keepin' On - Sylvie Cruz
Three Drives - Greece 2000 - Sylvie Cruz
Sebastien Tellier - Kilometer - Sylvie Cruz
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz


## Task3: Getting the user names by songs listened 
For this query the following columns are considered: song, userId, firstName, lastName.
Song and userId are used as COMPOSITE PRIMARY KEY, where song is used as a PARTITION KEY and userId is a CLUSTERING COLUMN.
FirstName and lastName columns are used to retrieve necessary data to satisfy the query.

In [103]:
query = "DROP TABLE IF EXISTS user_name_by_song"

try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE user_name_by_song" 
query = query + """(
                     song text,
                     userId int,
                     firstName text, 
                     lastName text,
                     PRIMARY KEY (song, userId)
                     )
                """

try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_name_by_song 
                                        (
                                            song,
                                            userId,
                                            firstName,
                                            lastName
                                            )
                                        """
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],  int(line[10]), line[1], line[4]))
                    

In [104]:
query = "SELECT firstname, lastname FROM user_name_by_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Droping the tables before closing out the sessions

In [29]:
query = "DROP TABLE IF EXISTS app_logs_1"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_name_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()